In [147]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [148]:
# Step 1: Set a User-Agent to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Step 2: Initialize a session
session = requests.Session()
session.headers.update(headers)

# Step 3: Fetch the main page containing article links
base_url = "https://www.hyundainews.com"
query_result_url = base_url + "/en-us/releases?category_ids=382&categories=sales+releases&page_title=sales_releases"
response = session.get(query_result_url)


In [149]:
# Check for a successful response (200 OK)
if response.status_code == 200:
    print("Successfully fetched the main page.")
else:
    print(f"Failed to fetch the main page. Status code: {response.status_code}")

Successfully fetched the main page.


In [150]:
base_url

'https://www.hyundainews.com'

In [151]:
# Step 4: Parse the main page with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find all the article links on the page (assuming they are <a> tags)
article_links = []
for link in soup.find_all('a', href=True):
    # Filter for links that seem to be article URLs (e.g., those containing "releases")
    if '/en-us/releases/' in link['href']:
        if "https:" not in link['href']:
            article_links.append(base_url + link['href'])

In [152]:
article_links

['https://www.hyundainews.com/en-us/releases/4414',
 'https://www.hyundainews.com/en-us/releases/4414',
 'https://www.hyundainews.com/en-us/releases/4414',
 'https://www.hyundainews.com/en-us/releases/4383',
 'https://www.hyundainews.com/en-us/releases/4383',
 'https://www.hyundainews.com/en-us/releases/4383',
 'https://www.hyundainews.com/en-us/releases/4362',
 'https://www.hyundainews.com/en-us/releases/4362',
 'https://www.hyundainews.com/en-us/releases/4362',
 'https://www.hyundainews.com/en-us/releases/4342',
 'https://www.hyundainews.com/en-us/releases/4342',
 'https://www.hyundainews.com/en-us/releases/4342',
 'https://www.hyundainews.com/en-us/releases/4322',
 'https://www.hyundainews.com/en-us/releases/4322',
 'https://www.hyundainews.com/en-us/releases/4322',
 'https://www.hyundainews.com/en-us/releases/4285',
 'https://www.hyundainews.com/en-us/releases/4285',
 'https://www.hyundainews.com/en-us/releases/4285',
 'https://www.hyundainews.com/en-us/releases/4258',
 'https://ww

In [153]:
# Step 3: Function to extract tables from an article
def extract_tables_from_article(url, total_sales_df, total_vehicle_sales_df):
    print(f"Extracting from {url}")
    # Fetch the article page
    article_response = session.get(url)
    article_soup = BeautifulSoup(article_response.text, 'html.parser')
    
    # Find all tables in the article
    tables = article_soup.find_all('table')
    
    # If there are tables, process them
    if tables:
        for idx, table in enumerate(tables):
            # Read the table into a pandas DataFrame
            df = pd.read_html(str(table))[0]
            prefix = ""

            # Remove any leading/trailing whitespaces in the cells
            df.iloc[0, 1] = df.iloc[0, 1].replace(" ", "")
            
            if idx == 0:  # Total Sales Table
                timestamp_str = df.iloc[0, 1]  # Row 0, Col 1
                timestamp = pd.to_datetime(timestamp_str, format='%b-%y')
                total_sales = df.iloc[1, 1]  # Assuming sales values are in the second column (adjust if needed)

                new_df = pd.DataFrame({
                    'timestamp': [timestamp],
                    'total sales': [total_sales]
                })

                # Append to the total_sales_df DataFrame
                total_sales_df = pd.concat([total_sales_df, new_df], ignore_index=True)          
            
            else:  # Vehicle Sales Table
                timestamp_str = df.iloc[0, 1]  # Row 0, Col 1
                timestamp = pd.to_datetime(timestamp_str, format='%b-%y')                
                new_df = pd.DataFrame({
                    'timestamp': [timestamp],
                })                   
                
                for i in range(1, len(df[0])):  # Assuming first column contains vehicle models and the second contains the sales data
                    new_df[df.iloc[i, 0]] = df.iloc[i, 1]
                
                # Append to the total_vehicle_sales_df DataFrame
                total_vehicle_sales_df = pd.concat([total_vehicle_sales_df, new_df], ignore_index=True)   
    else:
        print(f"No tables found in {url}.")
    
    return total_sales_df, total_vehicle_sales_df  # Return updated DataFrames



In [154]:
# Step 4: Loop through article links and extract tables

total_sales_df = pd.DataFrame(columns=['timestamp', 'total sales'])
total_vehicle_sales_df = pd.DataFrame(columns=['timestamp'])


for article_url in article_links:
    total_sales_df, total_vehicle_sales_df = extract_tables_from_article(article_url, total_sales_df, total_vehicle_sales_df)

# drop duplicates
total_sales_df = total_sales_df.drop_duplicates()
total_vehicle_sales_df = total_vehicle_sales_df.drop_duplicates()

# Save the table as a CSV file
total_sales_df.to_csv(f"data/total_sales_df.csv", index=False)    
total_vehicle_sales_df.to_csv(f"data/total_vehicle_sales_df.csv", index=False)    
print("Extraction complete.")

Extracting from https://www.hyundainews.com/en-us/releases/4414
Extracting from https://www.hyundainews.com/en-us/releases/4414
Extracting from https://www.hyundainews.com/en-us/releases/4414
Extracting from https://www.hyundainews.com/en-us/releases/4383


C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_sales_df = pd.concat([total_sales_df, new_df], ignore_index=True)
C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Hong-

Extracting from https://www.hyundainews.com/en-us/releases/4383
Extracting from https://www.hyundainews.com/en-us/releases/4383
Extracting from https://www.hyundainews.com/en-us/releases/4362


C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To

Extracting from https://www.hyundainews.com/en-us/releases/4362
Extracting from https://www.hyundainews.com/en-us/releases/4362
Extracting from https://www.hyundainews.com/en-us/releases/4342
Extracting from https://www.hyundainews.com/en-us/releases/4342


...

C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To

Extracting from https://www.hyundainews.com/en-us/releases/3560
Extracting from https://www.hyundainews.com/en-us/releases/3560
Extracting from https://www.hyundainews.com/en-us/releases/3530
Extracting from https://www.hyundainews.com/en-us/releases/3530


C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To

Extracting from https://www.hyundainews.com/en-us/releases/3516
Extracting from https://www.hyundainews.com/en-us/releases/3516
Extracting from https://www.hyundainews.com/en-us/releases/3497
Extracting from https://www.hyundainews.com/en-us/releases/3497
Extracting from https://www.hyundainews.com/en-us/releases/3475
Extracting from https://www.hyundainews.com/en-us/releases/3475
Extraction complete.


C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\Hong-Bin\AppData\Local\Temp\ipykernel_9820\3232788066.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To